# Project: Disaster Response Pipeline App

## ETL Pipeline Preparation For Disaster Messages Classification
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import necessary libraries 
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
import pickle

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
with open('../password.txt') as f:
    password = f.read().strip()

In [3]:
# load data
connection_string = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:serverxx.database.windows.net,1433;Database=DB_LAB04;Uid=tendangnhap;"+f"Pwd={password}"+";Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={connection_string}")
df = pd.read_sql_table('Disaster_Response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## ML Pipeline Preparation For Disaster Messages Classification
Follow the instructions below to help you create your ML pipeline.


In [14]:
# split the dataset
X = df.message
y = df.iloc[:,4:]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # detect all URL present in the messages
    detected_urls = re.findall(url_regex, text)
    # replace URL with "urlplaceholder"
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
    ('cvect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('cvect',
                 CountVectorizer(tokenizer=<function tokenize at 0x78e7cc9785f0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)
y_pred[123].shape

(36,)

In [20]:
# y_pred = pipeline.predict(X_test)

for i in range(36):
    print("=======================",y_test.columns[i],"======================")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

======================= ('related',) ======================
              precision    recall  f1-score   support

           0       0.73      0.25      0.37      1548
           1       0.80      0.97      0.88      4953
           2       1.00      0.02      0.04        53

    accuracy                           0.79      6554
   macro avg       0.84      0.41      0.43      6554
weighted avg       0.78      0.79      0.75      6554

======================= ('request',) ======================
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.86      0.44      0.59      1109

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

======================= ('offer',) ======================
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
     

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('cvect',
   CountVectorizer(tokenizer=<function tokenize at 0x78e7cc9785f0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'cvect': CountVectorizer(tokenizer=<function tokenize at 0x78e7cc9785f0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'cvect__analyzer': 'word',
 'cvect__binary': False,
 'cvect__decode_error': 'strict',
 'cvect__dtype': numpy.int64,
 'cvect__encoding': 'utf-8',
 'cvect__input': 'content',
 'cvect__lowercase': True,
 'cvect__max_df': 1.0,
 'cvect__max_features': None,
 'cvect__min_df': 1,
 'cvect__ngram_range': (1, 1),
 'cvect__preprocessor': None,
 'cvect__stop_words': None,
 'cvect__strip_accents': None,
 'cvect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'cvect__tokenizer': <function __main__.tokenize(text)>,
 'cvect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_t

In [22]:
parameters = {
    'clf__estimator__n_estimators': [ 100, 150],
    'clf__estimator__min_samples_split': [2, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2,)

In [23]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 4.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150, total= 6.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150, total= 6.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 101.4min finished


GridSearchCV(estimator=Pipeline(steps=[('cvect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x78e7cc9785f0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 4],
                         'clf__estimator__n_estimators': [100, 150]},
             verbose=2)

In [24]:
cv.cv_results_

{'mean_fit_time': array([255.02370491, 381.44016085, 214.73059535, 319.75268292]),
 'std_fit_time': array([1.09550442, 2.5053993 , 0.89526991, 1.66163705]),
 'mean_score_time': array([ 9.4991972 , 13.55894756,  9.42673316, 13.54076457]),
 'std_score_time': array([0.04169446, 0.06572034, 0.0425771 , 0.04401747]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 4, 4],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[100, 150, 100, 150],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 100},
  {'clf__estimator__min_samples_split': 2,
   'clf__estimator__n_estimators': 150},
  {'clf__estimator__min_samples_split': 4,
   'clf__estimator__n_estimators': 100},
  {'clf__estimator__min_samples_split': 4,
   'clf__estimator__n_es

In [25]:
#finding the best paramesters based on grip search
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 150}


In [26]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(steps=[('cvect',
                 CountVectorizer(tokenizer=<function tokenize at 0x78e7cc9785f0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=150)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
y_pred = optimised_model.predict(X_test)

for i in range(36):
    print("=============================",y_test.columns[i], '=================================')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

============================= ('related',) =================================
              precision    recall  f1-score   support

           0       0.72      0.25      0.37      1548
           1       0.80      0.97      0.88      4953
           2       1.00      0.02      0.04        53

    accuracy                           0.79      6554
   macro avg       0.84      0.41      0.43      6554
weighted avg       0.78      0.79      0.75      6554

============================= ('request',) =================================
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.87      0.43      0.58      1109

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

============================= ('offer',) =================================
              precision    recall  f1-score   support

        

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [29]:
pickle.dump(optimised_model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.